# Esercitazione di Tecnologie del Linguaggio Naturale - Parte Prima (prof. Mazzei)

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

## 1A. Algoritmo CKY

### Strutture dati e rappresentazione

Per l'implementazione dell'algoritmo abbiamo in primo luogo definito la tipologia di struttura dati sulla quale CKY dovrà operare. Nello specifico, abbiamo definito la tipologia di cella della
matrice (triangolare) che utilizzerà CKY: il contenuto di ogni cella deve permettere di tener traccia delle varie regole considerate dall'algoritmo.


Ciò è fondamentale poiché ci permetterà in seguito di fare backtracking e ottenere così le realizzazioni dei vari alberi di parsificazione.
Per questa ragione abbiamo definito ogni cella della matrice come una lista di `Entry`. Il tipo `Entry` è un [tipo somma](https://en.wikipedia.org/wiki/Tagged_union) che può essere `Terminal` oppure `NonTerminal`. Il primo rappresenta una regola che ha come conseguente un terminale, mentre il secondo rappresenta una regola con due non terminali come conseguente (queste sono infatti le uniche due tipologie di regole ammesse in grammatica in *Chomsky Normal Form*, prerequisito di CKY). Questi due non terminali sono dei puntatori associati a loro volta ad altre entry presenti all'interno della matrice. Come si vedrà in seguito, queste associazioni verranno poi costruite durante l'esecuzione dell'algoritmo.

In [27]:
# Each table entry will contain a list of Termina and NonTerminal
class Entry:
  def __init__(self, value):
    self.value = value

  def get_value(self):
    return self.value

  def is_start(self):
    return self.value == Nonterminal('S')

class Terminal(Entry):
  def __init__(self, value, word):
    self.value = value
    self.word = word

  def __repr__(self):
    return f'({self.value} {self.word})'

class NonTerminal(Entry):
  def __init__(self, value, left, right):
    self.value = value
    self.left = left
    self.right = right

  def __repr__(self):
    return f'({self.value} {self.left.value} {self.right.value})'

### Implementazione

Dopo aver definito la tipologia di struttura dati, è utile inoltre introdurre alcune funzioni di supporto che implementano alcune operazioni ad alto livello che compaiono nella definizione dell'algoritmo. Più nel dettaglio, definiamo due funzioni rispettivamente per:

1. Trovare nella grammatica tutte le regole consistenti con il terminale dato (`find_consistent_terminal`)
2. Trovare nella grammatica tutte le regole consistenti con due non terminali dati (`find_consistent_nonterminal`)

Si noti che l'implementazione di queste funzioni assume che le grammatiche siano rappresentate come `nltk.grammar.CFG`. In caso la grammatica fornita non sia di questo tipo, si otterrà un errore a runtime dovuto all'assenza di funzioni specifiche.

In [28]:
from typing import List
import nltk

def find_consistent_terminal(grammar: nltk.grammar.CFG, terminal: str) -> List[Terminal]:
  return map(lambda x: Terminal(x.lhs(), terminal), grammar.productions(rhs = terminal))

def find_consistent_nonterminal(grammar: nltk.grammar.CFG, b: NonTerminal, c: NonTerminal) -> List[NonTerminal]:
  result = list()

  for production in grammar.productions():
    if(len(production.rhs()) == 2):
      (b_i, c_i) = production.rhs()
      # Check for all possible entries in both sets
      for entry_b in b:
        for entry_c in c:
          if b_i == entry_b.get_value() and c_i == entry_c.get_value():
            result.append(NonTerminal(production.lhs(), entry_b, entry_c))

  return result

L'implementazione dell'algoritmo diventa a questo punto molto semplice, poiché segue direttamente dalla definizione in pseudocodice data dal libro. Si noti che nell'implementazione seguente gli indici sono stati rivisti poiché nello pseudocodice la lista delle parole inizia con l'indice $1$, mentre in Python con l'inidice $0$.

In [29]:
def cky_parse(words: List[str], grammar: nltk.grammar.CFG) -> List[List[Entry]]:

  table = [[[] for i in range(len(words))] for j in range(len(words))]

  for j in range(0, len(words)):
    table[j][j].extend(find_consistent_terminal(grammar, words[j]))
    for i in range(j, -1, -1):
      for k in range(i, j):
        table[i][j] += find_consistent_nonterminal(grammar, table[i][k], table[k+1][j])

  return table

Per definizione, l'algoritmo ritorna una tabella che rappresenta tutte le possibili derivazioni degli alberi di parsificazione. Si avranno tanti alberi tante quante saranno le produzioni con antecedente $S$ (*start symbol*) presenti nella cella alla posizione $[0,\: len(words)]$. Naturalmente, se tale produzione non compare in quella posizione, la grammatica non copre la frase. Possiamo sfruttare questo fatto per verificare se una frase sia coperta o meno dalla grammatica.

In [30]:
def is_covered(words, grammar):
  table = cky_parse(words, grammar)
  for entry in table[0][len(words) - 1]:
    if entry.is_start():
      return True
  return False

## 1B. Grammatica L1

Come accennato in precedenza, abbiamo deciso di utilizzare il pacchetto `grammar` della libreria `nltk` per rappresentare la grammatica. Per ottenere un'istanza della grammatica è sufficiente utilizzare la funzione di utility `nltk.grammar.CFG.fromstring`.

In [31]:
from nltk.grammar import CFG, read_grammar, Nonterminal

l1_grammar = CFG.fromstring(
"""
S -> NP VP
S -> X1 VP
X1 -> Aux NP
S -> "book" | "include" | "prefer"
S -> Verb NP
S -> X2 PP
S -> Verb PP
S -> VP PP
NP -> "I" | "she" | "me"
NP -> "TWA" | "Houston"
NP -> Det Nominal
Nominal -> "book" | "flight" | "meal" | "money" | "morning"
Nominal -> Nominal Noun
Nominal -> Nominal PP
VP -> "book" | "include" | "prefer"
VP -> Verb NP
VP -> X2 PP
X2 -> Verb NP
VP -> Verb PP
VP -> VP PP
PP -> Preposition NP

Det -> "that" | "this" | "the" | "a"
Noun -> "book" | "flight" | "meal" | "money" | "morning"
Verb -> "book" | "include" | "prefer"
Pronoun -> "I" | "She" | "Me"
Aux -> "does"
Preposition -> "from" | "to" | "on" | "near" | "through"
""")

Per coprire tutte le frasi assegnate (si veda la sezione successiva) si è dovuta espandere la grammatica fornita dal Jurafsky & Martin, nello specifico per la frase *does she prefer a morning flight*, dove la costruzione *morning flight* non veniva coperta dalla grammatica originale.

### Copertura

Avendo ottenuto la rappresentazione della grammatica nel formato corretto (`nltk.grammar.CFG`), è possibile ora testare la copertura delle frasi della consegna utilizzando la funzione da noi implementata `is_covered`. Testiamo anche l'algoritmo con una frase aggiuntiva non coperta dalla grammatica per verificarne ulteriormente la correttezza.

In [32]:
sentences = [
    "does she prefer a morning flight",
    "book the flight through Houston",
    "book the flight through Houston to book"
]

for sentence in sentences:
  result = is_covered(sentence.split(), l1_grammar)
  pretty_res = '\033[92mCOVERED' if result else '\033[91mNOT COVERED'
  print(f'[\033[1m{pretty_res}\033[0m]: \t \x1B[3m"{sentence}"\033[0m\n')

[COVERED]: 	 "does she prefer a morning flight"

[COVERED]: 	 "book the flight through Houston"

[NOT COVERED]: 	 "book the flight through Houston to book"



### Alberi di parsificazione

Precedentemente è stato discusso di come l'algoritmo CKY ritorni in forma tabellare tutti i possibili alberi di parsificazione. Grazie all'implementazione di `Entry` è possibile estrarre tutti gli alberi validi seguendo il percorso dei vari *backpointers* con una visita in profondità, partendo dai nonterminali $S$ presenti nella cella a coordinate $[0,\: len(words)]$.

In [33]:
def get_parsing_trees(words, grammar):

  parsing_trees = []
  table = cky_parse(words, grammar)

  for entry in table[0][len(words) - 1]:
    # Found an 'S' entry
    if entry.is_start():
      parsing_tree = []
      evaluation_stack = [entry]
      # Find the tree
      while evaluation_stack:
        current_node = evaluation_stack.pop(-1)
        match current_node:
          case NonTerminal(value = non_terminal, left = left_ptr, right = right_ptr):
            parsing_tree.append(current_node)
            evaluation_stack.append(left_ptr)
            evaluation_stack.append(right_ptr)
          case Terminal():
            parsing_tree.append(current_node)
      # Add the tree
      parsing_trees.append(parsing_tree)

  return parsing_trees

Sapendo che la funzione `get_parsing_trees` ritorna una lista di liste in cui ogni lista contiene i nodi di un albero di parsificazione valido, possiamo utilizzare una libreria quale `graphviz` per visualizzare graficamente un albero di parsificazione arbitrario.

In [34]:
import graphviz

def print_tree(parsing_tree):
  dot = graphviz.Graph()
  dot.attr('node', shape='none')

  # Add the nodes to the graph
  for node in reversed(parsing_tree):
    match node:
      case NonTerminal(value = non_terminal, left = left_ptr, right = right_ptr):
        dot.node(str(id(node)), str(non_terminal))
      case Terminal(value = terminal, word = word):
        dot.node(str(id(node)), str(terminal))
        dot.node(str(id(word)), str(word))

  # Add the edges between nodes to the graph
  for node in parsing_tree:
    match node:
      case NonTerminal(value = non_terminal, left = left_ptr, right = right_ptr):
        dot.edge(str(id(node)), str(id(left_ptr)))
        dot.edge(str(id(node)), str(id(right_ptr)))
      case Terminal(value = terminal, word = word):
        dot.edge(str(id(node)), str(id(word)))

  return dot

Utilizziamo ora la funzione definita per visualizzare graficamente il primo albero di parsificazione (in caso esso esista) su una frase specifica (*è possibile cambiare la frase considerata mediante l'apposito controllo. In caso non sia attivo, è necessario ri-valutare la cella seguente*).

In [35]:
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

def select_tree(tree):
  display(print_tree(tree))

def select_sentence(sentence):
  # get all parsing trees for the given sentence
  parsing_trees = get_parsing_trees(sentence.split(), l1_grammar)
  # if we have more than 0 parsing trees, show the corresponding control
  if not parsing_trees:
    print('No parse tree available')
  else:
    tree_index_control = interactive(select_tree, tree=list(enumerate(parsing_trees)))
    display(tree_index_control)

interactive(select_sentence, sentence=sentences)

interactive(children=(Dropdown(description='sentence', options=('does she prefer a morning flight', 'book the …

Si noti come per frasi sintatticamente ambigue, gli alberi di parsificazione sono molteplici.

## 1C. Grammatica Klingon

Proponiamo ora una grammatica per la lingua Klingon. La metodologia seguita à stata quella di partire creando inizialmente le regole terminali, per poi successivamente creare le regole di composizione grammaticale secondo il Klingon Dictionary.

Per questa ragione, la grammatica ottenuta è molto ristretta, poiché copre solamente le frasi fornite. Secondo una prima analisi, è stata quindi ottenuta la seguente Grammatica formale (non in CNF):

* $S \to \textrm{VP NP}$
* $S \to \textrm{VP Pronoun}$
* $S \to \textrm{NP VP}$
* $S \to \textrm{NP Pronoun}$
* $VP \to \textrm{NP VP}$
* $VP \to Dajatlh'a' \mid vIlegh \mid jIHtaH$
* $NP \to \textrm{Noun}$
* $NP \to \textrm{Noun} \: \textrm{Noun}$
* $NP \to \textrm{Pronoun}$
* $Noun \to pa'Daq \mid puq \mid tlhIngan \mid Hol$
* $Pronoun \to jIH \mid maH$

Rispetto all'Inglese, il Klingon è una lingua con *word-order* più frequente oggetto-verbo-soggetto, da questo la presenza di regole di riscrittura con VP come primo non-terminale (rispetto a quelle della grammatica L1 Inglese, dove si ha una maggior frequenza di NP come primo elemento).

Altro tratto caratteristico del Klingon è la mancanza del verbo *essere* (cfr. Klingon Dictionary, §6.3), dove però i pronomi possono essere usati come verbi. Così la frase *tlhIngan maH* (corrispondente all'Inglese *We are Klingon*) è composta semplicemente da un sostantivo e da un pronome. Sempre da questo esempio è possibile notare un'altra caratteristica tipica del Klingon: la mancanza di aggettivi.

Siccome CKY necessita però della grammatica in CNF, la riporteremo in questa forma seguendo la metodologia di trasformazione proposta dal libro.

In [36]:
klingon_grammar = CFG.fromstring(
"""
S -> VP NP
S -> VP Pronoun
S -> NP VP1
S -> NP Pronoun

VP -> NP VP1

NP -> "pa'Daq" | "puq" | "tlhIngan" | "Hol" | "jIH" | "maH"
NP -> Noun Noun

Noun -> "puq" | "tlhIngan" | "Hol"
Pronoun -> "jIH" | "maH"

VP1 -> "Dajatlh'a'" | "vIlegh" | "jIHtaH"
""")

Il terminale VP1 corrisponde ai verbi, si è scelta però questa denominazione rispetto a quella di *Verb* o *V* in quanto i verbi denotati da VP1 non sono semplici verbi ma contengono inoltre particelle e suffissi tipiche del Klingon.

Per sicurezza, verifichiamo inoltre che sia in CNF con la funzione apposita di `nltk`.

In [37]:
klingon_grammar.is_chomsky_normal_form()

True

### Copertura

Così come in precedenza, possiamo ora utilizzare CKY per verificare quali frasi siano coperte dalla grammatica attraverso la funzione `is_covered` da noi implementata.

In [38]:
sentences = [
    "tlhIngan Hol Dajatlh'a'", # Do you speak Klingon?
    "puq vIlegh jIH", # I see the child
    "pa'Daq jIHtaH", # I'm in the room
    "tlhIngan maH" # We are Klingon!
]

for sentence in sentences:
  result = is_covered(sentence.split(), klingon_grammar)
  pretty_res = '\033[92mCOVERED' if result else '\033[91mNOT COVERED'
  print(f'[\033[1m{pretty_res}\033[0m]: \t \x1B[3m"{sentence}"\033[0m\n')

[COVERED]: 	 "tlhIngan Hol Dajatlh'a'"

[COVERED]: 	 "puq vIlegh jIH"

[COVERED]: 	 "pa'Daq jIHtaH"

[COVERED]: 	 "tlhIngan maH"



### Alberi di parsificazione

Visualizziamo anche i vari alberi di parsificazione ottenuti come fatto in precedenza per la grammatica $L1$.

In [39]:
def select_tree(tree):
  display(print_tree(tree))

def select_sentence(sentence):
  parsing_trees = get_parsing_trees(sentence.split(), klingon_grammar)
  if not parsing_trees:
    print('No parse tree available')
  else:
    tree_index_control = interactive(select_tree, tree=list(enumerate(parsing_trees)))
    display(tree_index_control)

interactive(select_sentence, sentence=sentences)

interactive(children=(Dropdown(description='sentence', options=("tlhIngan Hol Dajatlh'a'", 'puq vIlegh jIH', "…